In [3]:
import findspark
findspark.init('/home/ubuntu/spark-2.2.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluser').getOrCreate()

In [8]:
#Import Data
folder = '/home/ubuntu/data/raw'
file = '/sample_kmeans_data.txt'
data = spark.read.format('libsvm').load(folder+file)
final_data = data.select('features')

In [9]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [13]:
##Create and fit model
from pyspark.ml.clustering import KMeans
km_clf = KMeans().setK(2).setSeed(1)
km_fitted = km_clf.fit(final_data)

In [18]:
#withing set sum quarederror
wssse = km_fitted.computeCost(final_data)
print(wssse)

centers = km_fitted.clusterCenters()
print(centers)

0.11999999999994547
[array([ 0.1,  0.1,  0.1]), array([ 9.1,  9.1,  9.1])]


In [20]:
#Find the cluster each datapoint falls on
km_fitted.transform(final_data).show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         0|
|(3,[0,1,2],[0.1,0...|         0|
|(3,[0,1,2],[0.2,0...|         0|
|(3,[0,1,2],[9.0,9...|         1|
|(3,[0,1,2],[9.1,9...|         1|
|(3,[0,1,2],[9.2,9...|         1|
+--------------------+----------+



In [21]:
#Try a higher k=3
km_clf2 = KMeans().setK(3).setSeed(1)
km_fitted2 = km_clf2.fit(final_data)
wssse2 = km_fitted2.computeCost(final_data)
print(wssse)

centers2 = km_fitted2.clusterCenters()
print(centers2)

#Find the cluster each datapoint falls on
km_fitted2.transform(final_data).show()

0.11999999999994547
[array([ 9.1,  9.1,  9.1]), array([ 0.05,  0.05,  0.05]), array([ 0.2,  0.2,  0.2])]
+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

